# Technical Report - "Beat the Streak" Hit Predictor
---
---

## Problem Statement
---
Since 2001, Major League Baseball has hosted the "Beat the Streak" competition. Every day of the regular season, participants can choose a player who they think will record a hit in that day's game. If a participant builds up a virtual "hitting streak" of 57 games to beat Joe DiMaggio's record 56-game hitting streak, he or she will receive a grand prize of $5.6 million dollars. With the competition currently in its nineteenth year, no one has ever won the prize.

In order to increase the likelihood of success in "Beat the Streak," I will use predictive analytics on batter performance data from three years worth of individual regular season games to create a binary classification model. This model will try to determine the safest player selections on a given day to extend a streak.

---
---
## Data Collection
---
For this project, I collected batting data for the 2015-2018 regular seasons from [FanGraphs](https://www.fangraphs.com "FanGraphs"), along with batting data for the beginning of the 2019 season (through May 12) . Each row of data represents a single batter's stats for a single game. I downloaded the data in a series of csv files, but because of the way the data is structured I needed to reassemble each season into a single pandas DataFrame.

FanGraphs categorizes the data into separate files for standard batting stats, advanced batting stats, and batted ball stats. A complete season of data consists of six files for each of these three different stat groups for a total of 18 csv files per season. I created a function to speed up the process of stitching together the 18 files into a single DataFrame for each season. The function takes in a base filepath that specifies the season and stat group, and it takes in the number of files in that series. It then concatenates all the files for each stat group.

Once the four comprehensive DataFrames were compiled, one per season, I moved on to EDA, cleaning, and pre-processing.

## EDA
The first step of my EDA and cleaning was to gauge the frequency of player appearances in each of the three seasons.

![full_player_freq](techreport_images/full_player_freq.png)

I didn't like what I saw in that distribution. The data was dominated by observations of players who each individually made very few appearances in the season. If a player played that infrequently, I couldn't assume that his skills were in line with those of an everyday player. A player with so few offensive appearances would also be more likely to be a pitcher, particularly in the American League where his batting opportunities would be limited to interleague away games. In any event, the performance of these infrequent batters was unlikely to be useful for predicting the performance of more offensive-focused players, and even the best bench players would represent a much riskier proposition as Beat the Streak picks, so being able to predict their performance was not a priority for this project. For these reasons, I dropped all observations for players who appear in less than one quarter of the season's games, leaving me with the following distribution:

![75_player_freq](techreport_images/75_player_freq.png)

I did a high-level examination of the distributions of the various X features and the results were not promising.

![eda_dist](techreport_images/eda_dist.png)

There were an overwhelming number of zero values within this data. My hope was that the minority of non-zero instances would provide signal to use in modeling.

To mitigate the abundance of zero values, I added a series of custom interaction features to the observations. These stats added trend elements to the data by calculating a number of 3- and 5-game rolling metrics for the number of plate appearances, the rate of balls in play per plate appearance, and the rate of hits per plate appearance.

I also used a time series shift to create the target y variable of whether or not each player would record a hit in their next game.

After all three seasons of data were collected into a single DataFrame and these custom stats were added, the X data used in modeling consisted of 131,208 observations and each observation would have approximately 50 numeric features on which to predict.

---
---
## Modeling
---

### Some Thoughts on Metric Selection
While I did check my models' accuracy scores throughout this process to get a general sense of model performance, I've chosen to optimize my models for precision. Within the context of the Beat the Streak contest, accuracy is far less important than precision. If my model doesn't pick up on the fact that a player is going to get a hit in their next game, then yes I have lost the opportunity to add to my streak. But in terms of value gained and lost, missing out on a streak extension because of a false negative (Type II error) is far less damaging to my goal than a false positive (Type I error), which would kill the current streak entirely and force me back to zero. For many years, not making a selection on any given day would automatically reset a participant's streak. But current Beat the Streak rules do not require a daily pick. That means that precision is even more valuable as the scoring metric for these models because I can afford to be highly selective in my picks. If none of the predicted probabilties on a given day look promising, I can abstain from making a pick and wait for what the model believes are safer picks. Hypothetically, if a highly-selective model has horrible accuracy from an abundance of false negatives but correctly predicts 57 hitting players with a precision score of 1 (meaning the predictions contain zero false positives), then that low-accuracy model could still be used to secure a $5.6 million payday.

---
### Logistic Regression
Logistic regression was a natural first approach to my modeling. While the mathematics behind it is simpler than other modeling methods, that simplicity allows it to fit a binary classification model much faster than decision trees or neural networks and its results are more interpretable.

I read in the full 2015-2018 dataset and assigned its numeric features to an X set and the `next_game_hit` values to a Y set as the model's target variable. I did not include the two features that focused on base-stealing performance in the X set. One could argue that base-stealing might serve as an indicator of running speed, which would have an impact on successfully reaching base on a hit ball. But I believe it could be just as indicative of the opposing catcher's defensive abilities (since fewer stolen bases are even attempted against the league's better catchers) or an opposing pitcher's ability to keep baserunners in check, which is a skill independent of quality pitch delivery.

I performed an unshuffled train-test split on the datasets because of the data's time-based origins and then fit the model. The initial accuracy scores were an unimpressive .616 on the training and .609 on the test, compared to a baseline accuracy/precision score of .590 if we were to predict "True" for all observations.

The distribution of predicted probabilites did not inspire much confidence. As shown below, they were all mostly grouped between .3 and .7:

![logreg_prob_dist](techreport_images/logreg_prob_dist.png)

That distribution showed that, assuming a decision threshold of .5, the model was predicting neither positive nor negative outcomes with any real certainty since we didn't see any probabilities anywhere near the extreme ends of the spectrum. Next I checked the model's actual performance visually.

![logreg_predprob50](techreport_images/logreg_predprob50.png)

This was not a promising result. The dashed red line indicates the .5 probability threshold. Anything to the right of that line was a prediction of "hit" from the model and anything to the left of the line indicates a prediction of "no hit," so the mass of blue on the positive side of this plot was discouraging. Though the negative predictions were shifted slightly more toward the negative side of the plot than the positive predictions were, they both had very similar distributions. I'd really like to see more separation between the two actual value distributions.

At this point, what I could do was optimize the model for precision, as demonstrated in the following visualization:

![logreg_predprob59](techreport_images/logreg_predprob59.png)

The same probabilities, when predicting on a shifted decision threshold of .59, made for predictions that were less accurate but more precise. The simple change of shifting the decision treshold allowed the predictions to pick up nearly three more points in the precision score. As explained above, the expense in accuracy for a precision-optimized model was an appropriate trade for this particular problem statement. By applying this model with a decision threshold of .59, I saw the precision score improve by roughly five percentage points over the baseline model.

Before moving on to different modeling methods I took advantage of linear regression's interpretability and visualized the most heavily-weighted features in the model.

![logref_coef_weights](techreport_images/logref_coef_weights.png)

According to this plot, the two features most influential features in making a positive prediction were Weighted On Base Average and Weighted Runs Created. The two most influential features in making a negative prediction were Weighted Runs Above Average and Weighted Runs Created. All of these stats are reasonable candidates to be among the most influential features because (in the case of wRC, wRAA, and wOBA) they reflect a comprehensive approach to measuring offensive performance and wRC+ takes venue and context into account. And wRAA is tied to generating runs as a hitter, so it makes sense that it would have great influence in the model. But it does concern me that wRC+ and wRAA were both exerting their strong influence in a negative direction rather than a positive one since the higher those stats are the more indicative of hitting prowess I would expect them to be.

#### Results

|Matrix|Actual Pos|Actual Neg|
|-|-|-|
|Pred. Pos|12525|7044|
|Pred. Neg|6848|6385|

The logistic regression model's performance left much to be desired, although I was able to optimize it for 5% improvement in precision over the baseline. Next I explored whether a decision tree would be more effective.

---
### Random Forest

Next I applied a random forest model to my data. I did this using a GridSearch with a range of values for the model's `n_estimators`, `max_depth`, `max_features`, and `min_samples_split`. The best parameters as determined by the GridSearch were `max_depth=6`, `max_features=10`, `min_samples_split=2`, and `n_estimators=175`.

Unfortunately, the results of the random forest were not an improvement over the logistic regression model, as shown below:

![rf_predprob50](techreport_images/rf_predprob50.png)

The distributions of the random forest model's predicted probabilities were actually very similar to those for the logistic regression model. I applied a similar shift to the decision threshold to optimize the model for precision.

![rf_predprob60](techreport_images/rf_predprob60.png)

The random forest model, particularly with an exhaustive GridSearch, is much more computationally intensive than logistic regression and not nearly as easily interpreted. For those reasons in addition to the lack of improved performance, I will move on to another modeling method.

#### Results
|Matrix|Actual Pos|Actual Neg|
|-|-|-|
|Pred. Pos|12565|7095|
|Pred. Neg|6808|6334|

---
### Neural Network

I next fit a sequential neural network to my data, but despite the increased computational needs, runtime, and model complexity, the performance was similar to the earlier models. Although the model did not seem to be particularly overfit, I attempted to get a better result by applying the regularization method of a dropout layer to a second neural network, the results of which are visualized below.

![nn_precision](techreport_images/nn_precision.png)

The network's performance against the testing data varied wildly, and its ultimate predictive performance should be very visually familiar by this point.

![nn_predprob60](techreport_images/nn_predprob60.png)

Neither neural network represents a substantial improvement over the logistic regression model. Similar to the random forest model, the lack of an upgrade in precision performance does not justify moving forward with the neural networks at this time.

#### Results
|Matrix|Actual Pos|Actual Neg|
|-|-|-|
|Pred. Pos|12131|6807|
|Pred. Neg|7242|6622|

---
### Polynomials & PCA

In a final attempt to find some meaningful signal, I applied polynomial transformations to my X data to add a broad range of feature interactions to the data, and I applied principal component analysis to offset the fact that there were then over 1300 features in the X dataset.

The polynomial and PCA transformations did make for a meaningful difference in the predicted probabilities compared to my earlier models. For the first time, I saw the tails of the predicted probabilties stretch out of the middle range and toward the extremes. Unfortunately, this newfound confidence extended to so few of the observations that the overall results of the model were again in line with the earlier models.

![poly_pca_predprob](techreport_images/poly_pca_predprob.png)

#### Results
|Matrix|Actual Pos|Actual Neg|
|-|-|-|
|Pred. Pos|12869|7332|
|Pred. Neg|6504|6097|

---
---
## Modeling Conclusions
---
Unfortunately, the primary takeaway from my investigation was that my data, in its current form, does not contain a great deal of real signal. For that reason, this broad range of modeling techniques are all returning similar results with muddled predicted probabilties. The lack of signal may also explain why throughout all of the various modeling methods, I never ran into significant overfitting in my models. Even the random forest model and neural network without regularization methods did not seem to have the overfitting I've often seen from those types of models. The dropoff in scores from training to testing sets were always reasonable, which tells me that the data did not have the requisite signal upon which to overfit.

I will discuss my next steps in the following section, but the model that I will consider my "final" model at this time is the logistic regression model with a .59 decision threshold. Although a couple of other models returned results that were technically better than this model's, the difference was always minimal and certainly not enough to outweight logistic regression's advantages of simplicity and interpretability. This model will serve as the foundation for my next steps.

---
---
## Moving Forward
---
The project as it stands is a reasonable first step but I can greatly improve the hit predictor in future iterations. Moving into Phase Two, I believe the biggest and most needed improvement is to **add opposing pitcher data** into the model's features. The myriad factors both in and out of each individual game that determine bullpen use will limit my ability to effectively include relief pitcher stats, but introducing relevant stats for each game's opposing starting pitcher is a reasonable next step. Introducing categorical **venue and weather statistics** for each game is the other main priority for Phase Two.

Phase Three of the hit predictor will be more advanced as it involves adding a clustering component. By analyzing data on players' physical attributes, situational player performance, and style of play, I can **create player types for both batters and starting pitchers**. By introducing these profiles to the model, I can account for how certain types of batters fare against certain types of pitchers and incorporate that interaction into the predictions.

Beyond the next steps of improving the hit predictor, if the process shows potential for broader performance prediction, these investigations could serve as the foundation of a **daily fantasy baseball lineup generator** for use on websites like DraftKings or FanDuel.

---
---

## Data Dictionary
Descriptions for stats from the standard, advanced, and batted ball datasets were sourced from FanGraphs.

|Feature|Full Name|Type|Dataset|Description|
|-|-|-|-|-|
|Date|Game Date|object|n/a|The date on which the game was played.|
|Name|Player Name|object|n/a|First and last name of the player.|
|Tm|Player's Team|object|n/a|Name of the player's team.|
|G  |Games Played|int|standard|Number of games in which the player has appeared.|
|AB |At Bats|int|standard|Number of trips to the plate in which the batter does not walk, get hit by a pitch, sacrifice (fly or bunt), or reach on interference.|
|PA |Plate Appearances|int|standard|Number of times the player has come to the plate.|
|H  |Hits|int|standard|Number of hits.|
|1B |Singles|int|standard|Number of singles.|
|2B |Doubles|int|standard|Number of doubles.|
|3B |Triples|int|standard|Number of triples.|
|HR |Home Runs|int|standard|Number of home runs.|
|R  |Runs Scored|int|standard|Number of runs scored.|
|RBI|Runs Batted In|int|standard|Number of times a run scores as a result of a batter’s plate appearance, not counting situations in which an error caused the run to score or the batter hit into a double play.|
|BB |Walks|int|standard|Total number of walks (includes IBB).|
|IBB|Intentional Walks|int|standard|Number of times the batter was intentionally walked.|
|SO |Strikeouts|int|standard|Number of strikeouts.|
|HBP|Hit By Pitches|int|standard|Number of times the batter reached after being hit by a pitch.|
|SF |Sacrifice Flies|int|standard|Number of times a batter’s fly out allowed a runner to tag up and score.|
|SH |Sacrifice Bunts|int|standard|Any bunt in which there was a runner on base and less than two outs in which the batter was put out and at least one runner advanced.|
|GDP|Grounded into Double Play|int|standard|Number of times the batter hit into a double play.|
|SB |Stolen Bases|int|standard|Number of stolen bases.|
|CS |Caught Stealing|int|standard|Number of times caught stealing.|
|AVG|Batting Average|float|standard|Rate of hits per at bat, calculated as H/AB.|
|playerId|n/a|int|n/a|Unique identifying number for each player.|
|BB% |Walk Percentage|float|advanced| Frequency with which the batter has walked, calculated as walks divided by plate appearances.|
|K% |Strikeout Percentage|float|advanced| Frequency with which the batter has struck out, calculated as strikeouts divided by plate appearances.|
|BB/K |Walk to Strikeout Rate|float|advanced| Ratio of walks to strikeouts, calculated as Walks/Strikeouts.|
|OBP |On Base Percentage|float|advanced| Rate at which the batter reaches base, calculated as (H+BB+HBP)/(AB+BB+HBP+SF).|
|SLG |Slugging Percentage|float|advanced| Average number of total bases per at bat, calculated as Total Bases/AB.|
|OPS |On Base Plus Slugging|float|advanced| Combination of OBP and SLG, calculated as OBP+SLG.|
|ISO |Isolated Power|float|advanced| Average number of extra bases per at bat, calculated several ways such as SLG minus AVG.|
|BABIP |Batting Average on Balls in Play|float|advanced| The rate at which the batter gets a hit when he puts the ball in play, calculated as (H-HR)/(AB-K-HR+SF).|
|wRC |Weighted Runs Created|float|advanced| Number of runs a player has generated for his team as a result of his wOBA and playing time.|
|wRAA |Weighted Runs Above Average|float|advanced| Number of runs above or below average a player has added as a hitter.|
|wOBA |Weighted On Base Average|float|advanced| Combines all the different aspects of hitting into one metric, weighting each of them in proportion to their actual run value. While batting average, on-base percentage, and slugging percentage fall short in accuracy and scope, wOBA measures and captures offensive value more accurately and comprehensively.|
|wRC+ |Weighted Runs Created Plus|float|advanced| The most comprehensive rate statistic used to measure hitting performance because it takes into account the varying weights of each offensive action (like wOBA) and then adjusts them for the park and league context in which they took place.|
|GB/FB|Ground Ball to Fly Ball Ratio|float|batted ball|The ratio of ground balls a batter hits to fly balls, calculated as GB/FB.|
|LD%  |Line Drive Percentage|float|batted ball|The percentage of a batter’s balls in play that are line drives, calculated as LD/BIP.|
|GB%  |Ground Ball Percentage|float|batted ball|The percentage of a batter’s balls in play that are ground balls, calculated as GB/BIP.|
|FB%  |Fly Ball Percentage|float|batted ball|The percentage of a batter’s balls in play that are fly balls, calculated as FB/BIP.|
|IFFB%|Infield Fly Ball Percentage|float|batted ball|Percentage of a batter’s fly balls that were infield fly balls, calculated as IFFB/FB.|
|HR/FB|Home Run to Fly Ball Rate|float|batted ball|Percentage of a batter’s fly balls that go for home runs, calculated as HB/FB (even though some HR are line drives).|
|IFH% |Infield Hit Percentage|float|batted ball|Percentage of ground balls that are infield hits, calculated as IFH/GB.|
|BUH% |Bunt Hit Percentage|float|batted ball|Percentage of bunts that go for hits, calculated as Bunt Hits/Bunts.|
|Pull%|Pull Percentage|float|batted ball|Percentage of batted balls hit to the pull field.|
|Cent%|Center Percentage|float|batted ball|Percentage of batted balls hit to the middle of the field.|
|Oppo%|Opposite Field Percentage|float|batted ball|Percentage of batted balls hit to the opposite field.|
|Soft%|Soft Contact Percentage|float|batted ball|Percentage of soft-hit batted balls.|
|Med% |Medium Contact Percentage|float|batted ball|Percentage of medium-hit batted balls.|
|Hard%|Hard Contact Percentage|float|batted ball|Percentage of hard-hit batted balls.|
|prev3_pa|n/a|float|custom|Player's rolling three-day sum number of plate appearances.|
|prev5_pa|n/a|float|custom|Player's rolling five-day sum number of plate appearances.|
|prev3_bippa|n/a|float|custom|Player's rolling three-day sum number of plate appearances that resulted in a ball in play (no walks or strikeouts) divided by his total number of plate appearances in that span.|
|prev5_bippa|n/a|float|custom|Player's rolling five-day sum number of plate appearances that resulted in a ball in play (no walks or strikeouts) divided by his total number of plate appearances in that span.|
|hpa|n/a|float|custom|Player's number of hits divided by his number of plate appearances in that day's game.|
|prev3_hpa|n/a|float|custom|Player's rolling three-day sum number of hits divided by his number of plate appearances in that span.|
|prev5_hpa|n/a|float|custom|Player's rolling five-day sum number of hits divided by his number of plate appearances in that span.|
|next_game_hit|n/a|bool|custom|A binary indicator of whether or not the player will record a hit in his next game. Used as the target y variable.|